In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
import numpy as np

# Daten laden
data_train = "../0_DataPreparation/final_data_train.csv"
df_train = pd.read_csv(data_train)

data_test = "../0_DataPreparation/final_data_test.csv"
df_test = pd.read_csv(data_test)

# Umwandeln der Datumsspalte in ein Datetime-Format
df_train['Datum'] = pd.to_datetime(df_train['Datum'])

# Datensatz nach Datum und Warengruppe sortieren
df_train_sorted = df_train.sort_values(by=['Datum', 'Warengruppe'])

# Features und Ziel definieren
feature_to_drop = ['id', 'Umsatz', 'Datum', 'Warengruppe', 'Bewoelkung', 'Temperatur',
                   'Windgeschwindigkeit', 'Wettercode', 'Wochentag',
                   'Monat', 'Jahr', 'Jahreszeit', 'Gefühl', 'InflationSensitivity', 'Windkategorie',  
                   'Tag_Kategorie', 'Inflation_Kategorisierung',
                   'Montag', 'Dienstag', 'Mittwoch',
                   'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
feature_to_drop2 = ['id', 'Datum', 'Warengruppe', 'Bewoelkung', 'Temperatur',
                   'Windgeschwindigkeit', 'Wettercode', 'Wochentag',
                   'Monat', 'Jahr', 'Jahreszeit', 'Gefühl', 'InflationSensitivity', 'Windkategorie',  
                   'Tag_Kategorie', 'Inflation_Kategorisierung',
                   'Montag', 'Dienstag', 'Mittwoch',
                   'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']

# Datenaufteilung in X (Features) und y (Target)
X = df_train_sorted.drop(columns=feature_to_drop)  # Entfernen der spezifischen Spalten
y = df_train_sorted['Umsatz']  # Ziel: Umsatz

df_test = df_test.drop(columns=feature_to_drop2)
print(X.info())

# Skalierung der Daten
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Aufteilen in Training (80%) und Validierung (20%) ohne Zufall (shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Eingabedimension
input_dim = X_train.shape[1]

# Modell definieren
model = Sequential()

# Input Layer und Hidden Layer 1
model.add(Dense(64, input_dim=input_dim, activation='relu'))

# Hidden Layer 2
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Hidden Layer 3
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Output Layer
model.add(Dense(1, activation='linear'))

# Modell kompilieren
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping]
)

# Loss-Kurve plotten
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Vorhersagen für die Trainings- und Validierungsdaten erstellen
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

# Berechnung des R²-Werts für Trainings- und Validierungsdaten
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Train R²: {train_r2:.4f}")
print(f"Validation R²: {val_r2:.4f}")

# Berechnung des MPAE
train_mpae = calculate_mpae(y_train, y_train_pred)
val_mpae = calculate_mpae(y_val, y_val_pred)

print(f"Train MPAE: {train_mpae:.4f}%")
print(f"Validation MPAE: {val_mpae:.4f}%")

# Berechnung des R²-Werts für Trainings- und Validierungsdaten
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Train R²: {train_r2:.4f}")
print(f"Validation R²: {val_r2:.4f}")

# Vorhersagen für df_test erstellen (ohne tatsächliche Umsatzwerte)
X_test_scaled = scaler.transform(df_test)  # Testdaten skalieren, wie es mit den Trainingsdaten gemacht wurde
y_pred = model.predict(X_test_scaled)

# Neue Spalte in df_test mit den Vorhersagen (Umsatz)
df_test['Umsatz_Vorhersage'] = y_pred

# Ausgabe der ersten Zeilen von df_test, um die Vorhersagen zu überprüfen
print(df_test[['Umsatz_Vorhersage']].head())

# Optional: Speichern des df_test mit den Vorhersagen
df_test.to_csv("../0_DataPreparation/final_data_test_with_predictions.csv", index=False)





